In [1]:
import pandas as pd
import sqlite3
from collections import defaultdict
import math
import re
from itertools import combinations 
import unidecode

In [2]:
con = sqlite3.connect('/Users/qlanners/projects/mlb_salary_comp_db/mlb_salary_comp.db')

In [3]:
inj_orig = pd.read_csv('../data/missedGamesInjuries.csv')

In [4]:
inj_updated = []
for p in list(set(inj_orig[inj_orig['Relinquished'].notnull()]['Relinquished'])):
# for p in ['Jason Varitek']:
    inj_player = inj_orig[(inj_orig['Relinquished'] == p) | (inj_orig['Acquired'] == p)].reset_index()
    inj_player_len = inj_player.shape[0]
    for i,v in inj_player.iterrows():
        if isinstance(v['Relinquished'], str):
            if inj_player_len > i+1:
                next_row = inj_player.iloc[i+1]
                if isinstance(next_row['Acquired'], str) and (v['Date'].split('-')[0] in next_row['Date']):
                    inj_updated.append([p, v['Date'], v['Team'], next_row['Date'], next_row['Team'], v['Notes']])
                else:
                    inj_updated.append([p, v['Date'], v['Team'], None, None, v['Notes']])
            else:
                inj_updated.append([p, v['Date'], v['Team'], None, None, v['Notes']])
                

In [5]:
col_names = ['name', 'injury_start', 'inj_start_team', 'inj_end', 'inj_end_date', 'inj_notes']
inj_updated_df = pd.DataFrame(inj_updated, columns=col_names)
# for some reason, one row is Nationals for name...
inj_updated_df = inj_updated_df[inj_updated_df['name'] != 'Nationals']

In [6]:
name_to_id = {n:None for n in inj_updated_df['name'].unique()}
total_players = len(name_to_id)

In [7]:
cur = con.cursor()

for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        year = int(r.injury_start.split("-")[0])
        cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%" and year == {year}')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]

unmatched_round_1 = [n for n,v in name_to_id.items() if v is None]
name_team_year_matches = len([n for n,v in name_to_id.items() if v is not None])
print(f'{name_team_year_matches} of {total_players} matches on name, team, and year. ({round((name_team_year_matches / total_players)*100, 2)}%)')


2059 of 2182 matches on name, team, and year. (94.36%)


In [8]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%"')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%"')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_2 = [n for n,v in name_to_id.items() if v is None]
name_team_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_1])
print(f'{name_team_matches} of {total_players} matches on name and team. ({round((name_team_matches / total_players)*100, 2)}%)')
print('Players matched w/o year:')
print([p for p in unmatched_round_1 if p not in unmatched_round_2])
                  

47 of 2182 matches on name and team. (2.15%)
Players matched w/o year:
['Mike Lincoln', 'Boof Bonser', 'Roy Alvarez / R.J. Alvarez', 'Justin Miller (Mark)', 'Brandon Cumpton', 'Tim Spooneybarger', 'Brandon Workman', 'Juan Perez (Pablo)', 'Nick Rumbelow', 'Juan Carlos Oviedo / Leo Nunez', 'Andrew McKirahan', 'Edgardo Alfonzo', 'Luis Marte', 'Andrew Brackman', 'Tyler Yates', 'Fernando Rodriguez Jr.', 'Philip Humber', 'Adam Plutko', 'Jose Ceda', 'Carl Sadler', 'Taylor Green', 'Logan Darnell', 'Brandon Claussen', 'Jerry Gil', 'Carter Capps', 'Chris Spurling', 'Kevin Hart', 'Jeff Zimmerman', 'John Rheinecker', 'Chris Pettit', 'Cory Wade', 'Casey Kelly', 'Marcos Mateo', 'Dan Winkler / Danny Winkler?', 'Scott Sullivan', 'John Franco (a)', 'Juan Padilla', 'Scott Elbert', 'Michael Pineda', 'Eric Surkamp', 'Josh Kinney', 'Calvin Pickering', 'Kyle Denney', 'Mike Zagurski', 'Humberto Sanchez', 'Tim Collins', 'Jairo Diaz']


In [9]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        year = int(r.injury_start.split("-")[0])
        cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and year == {year}')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_3 = [n for n,v in name_to_id.items() if v is None]
name_year_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_2])
print(f'{name_year_matches} of {total_players} matches on name and year. ({round((name_year_matches / total_players)*100, 2)}%)')
print('Players matched w/o team:')
print([p for p in unmatched_round_2 if p not in unmatched_round_3])

6 of 2182 matches on name and year. (0.27%)
Players matched w/o team:
['Ramon Cabrera', 'Ramon Laureano', 'Nick Green (Alvin)', 'Jose Reyes (Simon)', 'Michael Taylor (Anthony)', 'Steven Rodriguez (F.) / Paco Rodriguez']


In [10]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name_parts = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip().split()
        for p in combinations(name_parts, 2):
            name = ' '.join(p)
            year = int(r.injury_start.split("-")[0])
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]
                break
    if not name_to_id[r['name']] and '/' in r['name']:
        name_parts = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip().split()
        for p in combinations(name_parts, 2):        
            name = ' '.join(p)
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]
                break

unmatched_round_4 = [n for n,v in name_to_id.items() if v is None]
part_name_team_year_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_3])
print(f'{part_name_team_year_matches} of {total_players} matches on partial-name, team, and year. ({round((part_name_team_year_matches / total_players)*100, 2)}%)')
print('Players matched on part of name:')
print([p for p in unmatched_round_3 if p not in unmatched_round_4])


20 of 2182 matches on partial-name, team, and year. (0.92%)
Players matched on part of name:
['Albert Almora Jr.', 'Delino DeShields Jr.', 'Cesar Izturis Sr.', 'Damaso Marte Sr.', 'Vladimir Guerrero Sr.', 'Torii Hunter Sr.', 'Jose Cruz Jr.', 'Tony Armas Jr.', 'Pedro Florimon Jr.', 'Gary Matthews Jr.', 'Jose Vizcaino Sr.', 'Ken Griffey Sr.', 'Rickie Weeks Jr.', 'Fernando Tatis Sr.', 'John Mayberry Jr.', 'Roberto Hernandez Heredia', 'Tony Gwynn Sr.', 'Tony Gwynn Jr. / Anthony Gwynn', 'Sandy Alomar Jr.', 'Jerry Hairston Jr.']


In [11]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = unidecode.unidecode(re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip().replace('.',''))
        year = int(r.injury_start.split("-")[0])
        cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%" and year == {year}')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = unidecode.unidecode(re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip().replace('.',''))
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%" and year == {year}')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]

unmatched_round_5 = [n for n,v in name_to_id.items() if v is None]
clean_name_team_year_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_4])
print(f'{clean_name_team_year_matches} of {total_players} matches on cleaned-name, team, and year. ({round((clean_name_team_year_matches / total_players)*100, 2)}%)')
print('Players matched on unicode cleaned name:')
print([p for p in unmatched_round_4 if p not in unmatched_round_5])


8 of 2182 matches on cleaned-name, team, and year. (0.37%)
Players matched on unicode cleaned name:
['D.J. LeMahieu', 'Joshua Riddle / J.T. Riddle', 'Eduardo Núñez', 'Carlos Corporán', 'Héctor Rondón', 'J.B. Shuck', 'C.C. Sabathia', 'A.J. Ramos']


In [12]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip()
        cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%"')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip()
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%"')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_6 = [n for n,v in name_to_id.items() if v is None]
name_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_5])
print(f'{name_matches} of {total_players} matches on only name. ({round((name_matches / total_players)*100, 2)}%)')
print('Players matched on only name:')
print([p for p in unmatched_round_5 if p not in unmatched_round_6])

13 of 2182 matches on only name. (0.6%)
Players matched on only name:
['Burch Smith', 'Sean Nolin', 'Mauricio Robles', 'Pokey Reese', 'Reynaldo Garcia', 'Thomas Diamond', 'Jesse Biddle', 'Miguel Asencio', 'Matt Purke', 'Alex White', 'Kyle Skipworth', 'Antoan Richardson', 'Juan Gonzalez']


In [13]:
for i,r in inj_updated_df.iterrows():
    if not name_to_id[r['name']]:
        name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[0].strip()).strip().replace(' Jr.','').split()[-1]
        cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%"')
        rows = cur.fetchall()
        if len(rows) == 1:
            name_to_id[r['name']] = rows[0][0]
        elif '/' in r['name']:
            name = re.sub("[\(\[].*?[\)\]]", "", r['name'].split('/')[1].strip()).strip().replace(' Jr.','').split()[-1]
            cur.execute(f'SELECT DISTINCT(name_key) FROM meta WHERE name LIKE "%{name}%" and team LIKE "%{r.inj_start_team}%"')
            rows = cur.fetchall()
            if len(rows) == 1:
                name_to_id[r['name']] = rows[0][0]            

unmatched_round_7 = [n for n,v in name_to_id.items() if v is None]
last_name_team_matches = len([n for n,v in name_to_id.items() if v is not None and n in unmatched_round_6])
print(f'{last_name_team_matches} of {total_players} matches on only last name and team. ({round((last_name_team_matches / total_players)*100, 2)}%)')
print('Players matched on last name and team:')
print([p for p in unmatched_round_6 if p not in unmatched_round_7])

11 of 2182 matches on only last name and team. (0.5%)
Players matched on last name and team:
['Samuel Deduno', 'Jonathon Niese', 'Matt Joyce', 'Jean Rodriguez', 'Ivan DeJesus Jr. / Ivan De Jesus Jr.', 'Francisco Rosario', 'Jae-Weong Seo / Jae Seo', 'Hong-Chih Kuo', 'Justin Miller (Ryan)', 'Steve Tolleson', 'Jake Blalock']


In [14]:
print(f'{len(unmatched_round_7)} players left unmatched')
print('Unmatched players: ')
num = 1
for p in unmatched_round_7:
    print(f'\t{num}. {p}')
    num += 1

18 players left unmatched
Unmatched players: 
	1. Jon Shave
	2. Al Reyes
	3. Mark Merila
	4. Joe Maddon
	5. Alan Farina
	6. John Coppolella
	7. Dave Williams (Aaron)
	8. Ryan Westmoreland
	9. John Farrell (Edward)
	10. Kris Honel
	11. Jose Oquendo
	12. Roberto Hernandez
	13. Tony Cloninger
	14. Alfredo Silverio
	15. David Montgomery
	16. Zach Britton
	17. Jason Stumm
	18. Jason Kipnish


In [15]:
inj_updated_df.shape

(11032, 6)

In [16]:
inj_df = inj_updated_df[~inj_updated_df['name'].isin(unmatched_round_7)]
print(f'Dropped rows for unmatched players\nCut DF from {inj_updated_df.shape[0]} to {inj_df.shape[0]} rows. (-{inj_updated_df.shape[0]-inj_df.shape[0]})')

Dropped rows for unmatched players
Cut DF from 11032 to 11006 rows. (-26)


In [17]:
inj_df['name_key'] = ""
for i,r in inj_df.iterrows():
    r['name_key'] = name_to_id[r['name']]

/Users/qlanners/opt/anaconda3/envs/mlb_salary_comp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
inj_df.head()

,name,injury_start,inj_start_team,inj_end,inj_end_date,inj_notes,name_key
0,Craig Counsell,2006-04-22,Diamondbacks,2006-04-25,Diamondbacks,hamstring injury (DTD),counscr01
1,Craig Counsell,2006-05-07,Diamondbacks,2006-05-09,Diamondbacks,back injury (DTD),counscr01
2,Craig Counsell,2009-06-28,Brewers,2009-06-29,Brewers,hamstring injury (DTD),counscr01
3,Craig Counsell,2010-05-31,Brewers,2010-06-01,Brewers,quadriceps injury (DTD),counscr01
4,Jonny Venters,2012-05-06,Braves,2012-05-08,Braves,sore left shoulder (DTD),ventejo01


In [19]:
print('Job complete')

Job complete
